In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tensorflow.keras.applications import ResNet50

In [3]:
import numpy as np
from sklearn import metrics

In [85]:
from tensorflow.keras import Model, Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import load_model

In [12]:
import json
import os

from pprint import pprint

In [89]:
from data.coco.coco import CocoGenerator

In [97]:
from model.callbacks.metric_callbacks import MAPCallback
from model.callbacks.save_callback import SaveModel

# datasets

In [90]:
# train
train_dataset = CocoGenerator('train', '/share/DEEPLEARNING/datasets/mscoco')

In [91]:
generator_train = train_dataset.flow(batch_size=2)
X_train, Y_train = next(generator_train)

In [93]:
print(X_train.shape)
print(Y_train.shape)

(2, 224, 224, 3)
(2, 80)


In [94]:
# validation
val_dataset = CocoGenerator('val', '/share/DEEPLEARNING/datasets/mscoco')

In [95]:
print(len(val_dataset.image_ids_in_subset))

40504


In [96]:
batch_size = len(val_dataset.image_ids_in_subset)
generator_val = val_dataset.flow(batch_size=batch_size)
X_test, Y_test = next(generator_val)
print(X_test.shape, Y_test.shape)

(40504, 224, 224, 3) (40504, 80)


# model

In [86]:
# dropping the Dense(1000)
model = ResNet50(include_top=True, weights='imagenet')
x = model.layers[-2].output
x = Dense(train_dataset.nb_classes, activation='sigmoid', name='predictions')(x)
model = Model(inputs=model.input, outputs=x)

In [98]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [102]:
lr = 0.1
model.compile(loss='binary_crossentropy', optimizer=SGD(lr=lr), metrics=['binary_accuracy'])

# train

In [100]:
# callbacks
exp_folder = '/home/caleml/partial_experiments/exp_20190722_0030_TESTNB'

cb_list = list()
cb_list.append(SaveModel(exp_folder, 100))

map_cb = MAPCallback(X_test, Y_test, exp_folder)
cb_list.append(map_cb)

In [ ]:
batch_size=32
nb_epochs=20

steps_per_epoch_train = int(len(train_dataset.id_to_label) / batch_size) + 1
model.fit_generator(train_dataset.flow(batch_size=batch_size),
                    steps_per_epoch=steps_per_epoch_train,
                    epochs=nb_epochs,
                    callbacks=cb_list,
                    verbose=1)

Epoch 1/20
2586/2587 [============================>.] - ETA: 0s - loss: 0.0646 - binary_accuracy: 0.9778
Trying to save model @epoch=001 to /home/caleml/partial_experiments/exp_20190722_0030_TESTNB/model_100_001.h5
type true <class 'numpy.ndarray'>, type pred <class 'numpy.ndarray'>
ap scores type <class 'numpy.ndarray'>
interval evaluation - epoch: 0 - mAP score: 0.629046
2587/2587 [==============================] - 1924s 744ms/step - loss: 0.0646 - binary_accuracy: 0.9778
Epoch 2/20
2586/2587 [============================>.] - ETA: 0s - loss: 0.0585 - binary_accuracy: 0.9797
Trying to save model @epoch=002 to /home/caleml/partial_experiments/exp_20190722_0030_TESTNB/model_100_002.h5
type true <class 'numpy.ndarray'>, type pred <class 'numpy.ndarray'>
ap scores type <class 'numpy.ndarray'>
interval evaluation - epoch: 1 - mAP score: 0.651616
2587/2587 [==============================] - 1769s 684ms/step - loss: 0.0585 - binary_accuracy: 0.9797
Epoch 3/20
2586/2587 [====================

2586/2587 [============================>.] - ETA: 0s - loss: 0.0116 - binary_accuracy: 0.9965
Trying to save model @epoch=018 to /home/caleml/partial_experiments/exp_20190722_0030_TESTNB/model_100_018.h5
type true <class 'numpy.ndarray'>, type pred <class 'numpy.ndarray'>
ap scores type <class 'numpy.ndarray'>
interval evaluation - epoch: 17 - mAP score: 0.650712
2587/2587 [==============================] - 1443s 558ms/step - loss: 0.0116 - binary_accuracy: 0.9965
Epoch 19/20
 287/2587 [==>...........................] - ETA: 19:27 - loss: 0.0100 - binary_accuracy: 0.9972